In [ ]:
from pathlib import Path
import numpy as np
from airo_planner.utils import files
from pydrake.geometry import Meshcat
from pydrake.geometry import MeshcatVisualizer
from pydrake.planning import RobotDiagramBuilder
from cloth_tools.drake.building import finish_build
from pydrake.math import RigidTransform, RollPitchYaw
from airo_planner.visualization.drake_viz import add_meshcat_triad

In [ ]:
tcp_transform = np.identity(4)
tcp_transform[2, 3] = 0.175

In [ ]:
robot_diagram_builder = RobotDiagramBuilder()  # time_step=0.001 even when I set timestep I get the mimic joint warning
scene_graph = robot_diagram_builder.scene_graph()
plant = robot_diagram_builder.plant()
builder = robot_diagram_builder.builder()
parser = robot_diagram_builder.parser()

# Add visualizer
meshcat = Meshcat()
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

# Load URDF files
resources_root = str(files.get_resources_dir())
ur5e_urdf = Path(resources_root) / "robots" / "ur5e" / "ur5e.urdf"
robotiq_2f_85_gripper_urdf = Path(resources_root) / "grippers" / "2f_85_gripper" / "urdf" / "robotiq_2f_85_static.urdf"
table_urdf = "table.urdf"

arm_index = parser.AddModelFromFile(str(ur5e_urdf), model_name="arm_left")
gripper_index = parser.AddModelFromFile(str(robotiq_2f_85_gripper_urdf), model_name="gripper")
table_index = parser.AddModelFromFile(str(table_urdf))

# Weld some frames together
world_frame = plant.world_frame()
arm_frame = plant.GetFrameByName("base_link", arm_index)
arm_tool_frame = plant.GetFrameByName("tool0", arm_index)
gripper_frame = plant.GetFrameByName("base_link", gripper_index)
table_frame = plant.GetFrameByName("base_link", table_index)

X_W_B = RigidTransform(rpy=RollPitchYaw([0, 0, -np.pi / 2]), p=[0, 0, 0])

plant.WeldFrames(world_frame, arm_frame, X_W_B)
plant.WeldFrames(arm_tool_frame, gripper_frame, RigidTransform(rpy=RollPitchYaw([0, 0, np.pi / 2]), p=[0, 0, 0]))
plant.WeldFrames(world_frame, table_frame)

diagram, context = finish_build(robot_diagram_builder, meshcat)
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

In [ ]:
start_joints = np.deg2rad([0, -90, -90, -90, 90, 0])
plant.SetPositions(plant_context, arm_index, start_joints)
diagram.ForcedPublish(context)

In [ ]:
transform = RigidTransform(p=[0.25, 0, 0.4], rpy=RollPitchYaw([np.pi, 0, 0]))
tcp_pose_0 = np.ascontiguousarray(transform.GetAsMatrix4())

add_meshcat_triad(meshcat, "TCP Frame", X_W_Triad=transform)

with np.printoptions(precision=10, suppress=True):
    print("tcp_pose_0")
    print(tcp_pose_0)


In [ ]:
from ur_analytic_ik import ur5e

from cloth_tools.drake.visualization import publish_ik_solutions

# I have in Drake X_W_TCP and I want to get X_CB_TCP, to use X_CB_W for conversion
X_CB_B = RigidTransform(rpy=RollPitchYaw([0, 0, np.pi]), p=[0, 0, 0]).GetAsMatrix4() # 180 rotation between URDF base en control box base
X_CB_W = X_CB_B @ np.linalg.inv(X_W_B.GetAsMatrix4())
X_CB_TCP = X_CB_W @ tcp_pose_0

solutions = ur5e.inverse_kinematics_with_tcp(X_CB_TCP, tcp_transform)
publish_ik_solutions(solutions, 2.0, meshcat, diagram, context, arm_index)

In [ ]:
tcp_pose_1 = np.identity(4)
X = np.array([-1.0, 0.0, 0.0])
Y = np.array([0.0, 1.0, 0.0])
Z = np.array([0.0, 0.0, -1.0])
top_down_orientation = np.column_stack([X, Y, Z])
translation = np.array([-0.2, -0.2, 0.2])

tcp_pose_1[:3, :3] = top_down_orientation
tcp_pose_1[:3, 3] = translation

with np.printoptions(precision=3, suppress=True):
    print("tcp_pose_1")
    print(tcp_pose_1)

In [ ]:
add_meshcat_triad(meshcat, "tcp_pose_1", X_W_Triad=RigidTransform(tcp_pose_1))

In [ ]:
with np.printoptions(precision=3, suppress=True):
    print("inverse_kinematics")
    print(ur5e.inverse_kinematics(tcp_pose_1))


In [ ]:
solutions_2 = ur5e.inverse_kinematics_with_tcp(X_CB_W @ tcp_pose_1, tcp_transform)
publish_ik_solutions(solutions_2, 2.0, meshcat, diagram, context, arm_index)